# ⛹ push to github

In [89]:
#%%capture
import os
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive',
            force_remount=True
            )


notebookname = 'SEO.ipynb'

class github:
    def __init__(self, github_pat, github_email, github_username, github_repo, gdrive_notebook_folder, notebook_name):
        self.github_pat = userdata.get(github_pat)
        self.github_email = userdata.get(github_email)
        self.github_username = userdata.get(github_username)
        self.github_repo = userdata.get(github_repo)
        self.gdrive_notebook_folder = userdata.get(gdrive_notebook_folder)
        self.notebook_name = notebook_name

    def clone_repo(self):
        # Source file path in Google Drive
        source_file_path = f"/content/drive/MyDrive/{self.gdrive_notebook_folder}/{self.notebook_name}"

        # Repository details
        repo_url = f'https://{self.github_pat}@github.com/{self.github_username}/{self.github_repo}.git'

        # Clone the private repository
        !git clone {repo_url} cloned-repo
        os.chdir('cloned-repo')  # Switch to the cloned repository

        # Ensure the file exists in Google Drive
        if os.path.exists(source_file_path):
            # Copy the notebook into the cloned repository
            !cp "{source_file_path}" ./
        else:
            print(f"The file {source_file_path} was not found.")
            return  # Exit if the file doesn't exist

        # Git configuration
        !git config user.email "{self.github_email}"
        !git config user.name "{self.github_username}"

        # Add the file to Git
        !git add "{self.notebook_name}"

        # Commit the changes
        !git commit -m "Added {self.notebook_name} from Google Drive"

        # Push to the repository
        !git push origin main

        # Wechsle zurück ins übergeordnete Verzeichnis und lösche cloned-repo
        os.chdir('..')
        !rm -rf cloned-repo
        print("cloned-repo wurde wieder gelöscht.")



# Clone, add, and push the notebook
clone_2 = github('github_pat', 'github_email', 'github_username', 'github_repo_seo', 'gdrive_seo_folder', notebookname)
clone_2.clone_repo()


Mounted at /content/drive
Cloning into 'cloned-repo'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 45 (delta 12), reused 35 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 187.15 KiB | 4.68 MiB/s, done.
Resolving deltas: 100% (12/12), done.
[main a8066f0] Added SEO.ipynb from Google Drive
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite SEO.ipynb (96%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 10.25 KiB | 1.71 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/MarkH0705/SEO_Analyses.git
   d8a4e66..a8066f0  main -> main
cloned-repo wurde wieder gelöscht.


# 🕸 scrap

In [90]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import chardet


class WebsiteScraper:
    """
    Diese Klasse kümmert sich ausschließlich um das Sammeln und Extrahieren
    von Texten aus einer Website.
    """

    def __init__(self, start_url, max_pages=50):
        """
        :param start_url: Die Start-URL der Website, z.B. "https://www.example.com"
        :param max_pages: Maximale Anzahl Seiten, die gecrawlt werden.
        """
        self.start_url = start_url
        self.max_pages = max_pages

        # Hier speichern wir {URL: reiner_Text}
        self.scraped_data = {}

    def scrape_website(self):
        """
        Startet den Crawl-Vorgang, gefolgt von der Extraktion des Textes
        und dem Sammeln interner Links.
        """
        visited = set()
        to_visit = [self.start_url]
        domain = urlparse(self.start_url).netloc

        while to_visit and len(visited) < self.max_pages:
            url = to_visit.pop(0)
            if url in visited:
                continue
            visited.add(url)

            try:
                response = requests.get(url, timeout=10)

                # Rohdaten holen und Encoding per chardet bestimmen
                raw_data = response.content
                detected = chardet.detect(raw_data)
                encoding = "utf-8"
                text_data = raw_data.decode(encoding, errors="replace")

                # Nur weiterverarbeiten, wenn HTML-Content
                if (response.status_code == 200
                        and "text/html" in response.headers.get("Content-Type", "")):
                    soup = BeautifulSoup(text_data, "html.parser")

                    # Text extrahieren
                    text = self._extract_text_from_soup(soup)
                    self.scraped_data[url] = text

                    # Interne Links sammeln
                    for link in soup.find_all("a", href=True):
                        absolute_link = urljoin(url, link["href"])
                        if urlparse(absolute_link).netloc == domain:
                            if (absolute_link not in visited
                                    and absolute_link not in to_visit):
                                to_visit.append(absolute_link)

            except requests.RequestException as e:
                print(f"Fehler beim Abrufen von {url}:\n{e}")

    def _extract_text_from_soup(self, soup):
        """
        Extrahiert aus <p>, <h1>, <h2>, <h3>, <li> reinen Text,
        entfernt Script-/Style-/Noscript-Tags und Kommentare.
        """
        for script_or_style in soup(["script", "style", "noscript"]):
            script_or_style.decompose()

        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        texts = []
        for tag in soup.find_all(["p", "h1", "h2", "h3", "li", "faq4_qusestion", "faq4_answer"]):
            txt = tag.get_text(strip=True)
            if txt:
                texts.append(txt)

        return "\n".join(texts)




    def _extract_text_from_soup(self, soup):
        """
        Extrahiert aus <p>, <h1>, <h2>, <h3>, <li> reinen Text,
        aber NICHT die, die in .faq4_question oder .faq4_answer stecken.
        Außerdem extrahiert er separat die FAQ-Fragen und -Antworten
        (faq4_question / faq4_answer).
        """
        # 1) Script/Style entfernen
        for script_or_style in soup(["script", "style", "noscript"]):
            script_or_style.decompose()

        # 2) Kommentare entfernen
        from bs4 import Comment
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # 3) Normale Texte (p, h1, h2, h3, li), ABER nicht innerhalb von .faq4_question / .faq4_answer
        texts = []
        all_normal_tags = soup.find_all(["p", "h1", "h2", "h3", "li"])
        for tag in all_normal_tags:
            # Prüfen, ob das Tag einen Vorfahren (Parent) hat mit Klasse faq4_question oder faq4_answer
            # Wenn ja, ignorieren wir es
            if tag.find_parent(class_="faq4_question") or tag.find_parent(class_="faq4_answer"):
                continue

            txt = tag.get_text(strip=True)
            if txt:
                texts.append(txt)

        # 4) FAQ-Bereiche (Fragen + Antworten)
        #    a) Alle Frage-Elemente mit Klasse .faq4_question
        #    b) Alle Antwort-Elemente mit Klasse .faq4_answer
        #    Wir gehen davon aus, dass Frage i zum Antwort i passt.

        questions = soup.select(".faq4_question")
        answers = soup.select(".faq4_answer")

        # 5) Zusammenführen (Frage + Antwort)
        for q, a in zip(questions, answers):
            q_text = q.get_text(strip=True)
            a_text = a.get_text(strip=True)
            if q_text and a_text:
                combined = f"Frage: {q_text}\nAntwort: {a_text}"
                texts.append(combined)

        # 6) Als String zurückgeben
        return "\n".join(texts)





    def get_scraped_data(self):
        """
        Gibt das Dictionary {URL: Text} zurück.
        Du kannst damit arbeiten, Seiten filtern, etc.
        """
        return self.scraped_data


In [91]:
sc = WebsiteScraper(start_url="https://www.rue-zahnspange.de")

sc.scrape_website()
sc.get_scraped_data()

{'https://www.rue-zahnspange.de': 'RÜ\nModerne Zahnspangen für ein gesünderes Lächeln\nEntdecken Sie die Praxis RÜ\xa0Zahnspange und unsere vielseitigen Behandlungen für Kinder und Jugendliche, die ein langanhaltendes und gesundes Lächeln ermöglichen.\nÖffnungszeiten\nHier finden Sie uns\nAnbindungen\nVor unserer Tür befindet sich die Haltestelle Martinstraße, erreichbar über:\n\u200d\nBahn:\xa0107, 108, U11\nBus: 142, 160, 161\n\u200d\nDirekt gegenüber der Praxis befindet sich ein geräumiger Parkplatz.\nBehandlungen für ein perfektes Lächeln\nUnser Ziel ist es, Ihnen nicht nur ein ästhetisch ansprechendes Lächeln zu schenken, sondern auch Ihre gesamte Kiefergesundheit zu verbessern. Wir begleiten Sie auf jedem Schritt dieses Weges mit Professionalität und Sorgfalt.\nTransparente Aufklärung\nWir setzen auf offene Kommunikation und ausführliche Beratung, damit Sie jeden Schritt Ihrer Behandlung klar verstehen.\nSpezialisiert auf Kinder und Jugendliche\nUnser kinderfreundliches Team biet

# 🤖 chatbot

In [93]:
import openai
import time
os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_api_key')

class Chatbot:
    """
    Diese Chatbot-Klasse nutzt die neue Methode client.chat.completions.create()
    aus openai>=1.0.0 über openai.OpenAI().
    """

    def __init__(self, systemprompt, prompt):
        self.client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        self.systemprompt = systemprompt
        self.prompt = prompt
        self.context = [{"role": "system", "content": systemprompt}]
        self.model = "gpt-4o-mini-2024-07-18"  # Beispiel-Modell

    def chat(self):
        """
        Sendet den Prompt an das Chat-Interface und gibt den kompletten Antwort-String zurück.
        """
        self.context.append({"role": "user", "content": self.prompt})
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.context
            )
            response_content = response.choices[0].message.content
            self.context.append({"role": "assistant", "content": response_content})
            return response_content
        except Exception as e:
            print(f"Fehler bei der OpenAI-Anfrage: {e}")
            return ""


    def chat_with_streaming(self):
            """
            Interagiert mit OpenAI Chat Completion API und streamt die Antwort.
            """
            # Nachricht zur Konversation hinzufügen
            self.context.append({"role": "user", "content": self.prompt})

            # print(f"User: {self.prompt}")
            # print("AI: ", end="", flush=True)

            try:
                # Streaming-Option aktivieren
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=self.context,
                    stream=True
                )

                streamed_content = ""  # Zum Speichern der gestreamten Antwort

                for chunk in response:
                    # Debugging: Anzeigen, was tatsächlich in jedem Chunk enthalten ist
                    delta = chunk.choices[0].delta
                    content = getattr(delta, "content", "")

                    if content:  # Verarbeite nur nicht-leere Inhalte
                        print(content, end="", flush=True)
                        streamed_content += content

                print()  # Neue Zeile am Ende

                # Gestreamte Antwort zur Konversation hinzufügen
                self.context.append({"role": "assistant", "content": streamed_content})

                # Return the streamed content
                return streamed_content # This line was added

            except Exception as e:
                print(f"\nDEBUG: An error occurred during streaming: {e}")
                # Return empty string in case of error
                return "" # This line was added


# 🆎 NLP

In [94]:
def chunk_text(text, max_tokens=10000):
    """
    Teilt den Text in Blöcke auf, damit er nicht zu lang
    für die OpenAI-API wird.
    Hier sehr vereinfacht: 1 Token ~ 4 Zeichen.
    """
    chunks = []
    approx_char_limit = max_tokens * 4
    start = 0
    while start < len(text):
        end = start + approx_char_limit
        chunk = text[start:end]
        chunks.append(chunk)
        start = end
    return chunks


# 🚧 main SEO optimize

In [95]:
combined_analysis_list = []
filtered_urls = []

def main():
    # 1. SCRAPING
    start_url = "https://www.rue-zahnspange.de/"
    scraper = WebsiteScraper(start_url=start_url, max_pages=20)
    scraper.scrape_website()

    # Alle gescrapten Daten abrufen
    scraped_data = scraper.get_scraped_data()

    # 2. Sichten der Texte und Filtern
    #    Hier könntest du jetzt z. B. manuell prüfen, welche URLs wichtig sind.
    #    Wir geben einfach mal alle URLs aus:
    print("\n--- Gesammelte Seiten und Inhalte (gekürzt) ---")
    for url, text in scraped_data.items():
        print(f"\nURL: {url}")
        # Beispiel: Nur ersten 200 Zeichen zeigen
        print(f"Text: {text[:200]}...")




    EXCLUDED_KEYWORDS = ["impressum", "datenschutz", "agb"]

    # Alle URLs sammeln, die KEINEN der ausgeschlossenen Begriffe enthalten
    for url in scraped_data.keys():
        # Schauen, ob einer der EXCLUDED_KEYWORDS im URL-String (kleingeschrieben) vorkommt
        if any(keyword in url.lower() for keyword in EXCLUDED_KEYWORDS):
            # Falls ja, überspringen wir diese URL
            continue
        # Sonst nehmen wir sie auf
        filtered_urls.append(url)





    # 3. SEO-Analyse starten (für gefilterte Seiten)
    for url in filtered_urls:
        # Die gesamte Seite analysieren
        page_text = scraped_data[url]

        # 3.1 Chunken, um zu große Anfragen zu vermeiden
        text_chunks = chunk_text(page_text, max_tokens=10000)

        print(f"\n=== Analyzing {url} ===")
        all_analyses = []
        for i, chunk in enumerate(text_chunks):
            print(f" - Sende Chunk {i+1}/{len(text_chunks)} an Chatbot ...")

            # Prompt definieren (SEO)
            system_prompt = "Du bist ein hochqualifizierter SEO-Experte. Du hast Zahnmedizin und Kieferorthopädie studiert und arbeitest für erfolgreiche online marketing experten!"
            user_prompt = (
                "1. Untersuche den folgenden Text auf Keyword-Optimierung, Lesbarkeit und mögliche SEO-Verbesserungen. "
                "2. Formuliere den Text entsprechend optimaler SEO Sichtbarkeit um. Der Tonfall soll weiterhin nett und freundlich sein und einen warmen, einfühlsamen Eindruck machen. "
                "Die Fragen und Antworten in den FAQs sollen ebenfalls optimiert werden. Es dürfen keine Fragen oder sonstige Textabschnitte weggelassen werden! Wir wollen grundsätzlich den Text nicht substanzielle verändern, "
                "sondern nur eine bereits bestehende website optimieren."
                "3. Als Ausgabe gebe eineZusammen fassung der SEO Analyse wieder. Dazu code für einen pandas dataframe: In der ersten Spalte ist der original Text und in der zweiten Spalte ist deine optimierte version. in der dritten spalte gebe deine erläuterungen zur optimierung an! "
                "Benutze keine Formatierungszeichen wie '###', '#' oder '**'! "

                "Hier ist der Text chunk: \n\n"
                f"{chunk}"
            )

            # ChatGPT aufrufen
            cb = Chatbot(systemprompt=system_prompt, prompt=user_prompt)
            analysis = cb.chat_with_streaming()
            all_analyses.append(analysis)

            # Warte kurz (Rate Limits, API-Kosten etc.)
            time.sleep(1)

        # 3.2 Fertige Analyse (alle Chunks zusammen)
        combined_analysis = "\n".join(all_analyses)


        combined_analysis_list.append(combined_analysis)
        # print(f"\n--- SEO-Analyse für {url} ---")
        # print(combined_analysis)


if __name__ == "__main__":
    main()



--- Gesammelte Seiten und Inhalte (gekürzt) ---

URL: https://www.rue-zahnspange.de/
Text: RÜ
Moderne Zahnspangen für ein gesünderes Lächeln
Entdecken Sie die Praxis RÜ Zahnspange und unsere vielseitigen Behandlungen für Kinder und Jugendliche, die ein langanhaltendes und gesundes Lächeln e...

URL: https://www.rue-zahnspange.de/zahnspangen
Text: RÜ
Entdecken Sie unsere Zahnspangen
Jede Behandlung erfordert individuelle Zahnspangen verschiedenster Arten. Werfen Sie einen Blick darauf, was wir zu bieten haben.
Feste Zahnspangen
Verlässliche und...

URL: https://www.rue-zahnspange.de/behandlungsablauf
Text: RÜ
Wir begleiten Sie bei jedem Schritt
Entdecken Sie unseren umfassenden Behandlungsablauf, der jeden Schritt zu Ihrem idealen Lächeln sorgfältig begleitet.
Ihr Weg zu einem perfekten Lächeln
Bevor wi...

URL: https://www.rue-zahnspange.de/erwachsene
Text: RÜ
Verwandeln Sie Ihr Lächeln ohne Kompromisse
Entscheiden Sie sich für eines der weltweit beliebtesten Systeme mit transparenten

In [96]:
import re

def extract_seo_optimized_texts(analysis_list):
    """
    Sucht in jedem String der analysis_list nach Passagen unter der Überschrift
    'SEO optimierter Text' und gibt diese als Liste zurück.
    """
    # Regex-Pattern (mit Dotall-Flag (?s), damit '.' auch Zeilenumbrüche matcht):
    # - 'SEO optimierter Text' gefolgt von beliebigem Whitespace.
    # - anschließend erfassen wir (.*?), also den 'Text' bis zum Lookahead
    #   auf eine Zeile, die mit 'Erläuterungen' beginnt oder auf das String-Ende ($).
    pattern = re.compile(r"(?s)SEO optimierter Text\s*(.*?)(?=\nErläuterungen|$)")

    seo_texts = []

    for analysis_output in analysis_list:
        # Alle Fundstellen (falls mehrfach pro String)
        matches = pattern.findall(analysis_output)
        for match in matches:
            # Whitespace trimmen
            seo_text = match.strip()
            # Falls nötig, weitere Bereinigungen an seo_text vornehmen
            seo_texts.append(seo_text)

    return seo_texts

# Beispielaufruf:
if __name__ == "__main__":
    # Angenommen, du hast im main() bereits `combined_analysis_list` gefüllt:
    # combined_analysis_list = [ ... alle LLM-Outputs ... ]

    # Dann rufst du:
    seo_optimized_texts = extract_seo_optimized_texts(combined_analysis_list)

    # Jetzt enthält seo_optimized_texts ausschließlich die Inhalte unter
    # "SEO optimierter Text".
    # Dies kannst du nun für deinen RAG-Workflow (Punkt 1) verwenden.
    print("\n--- Extrahierte SEO-optimierte Texte ---")
    for i, seo_text in enumerate(seo_optimized_texts):
        print(f"\nOptimierter Text für {filtered_urls[i]}:\n{seo_text}")



--- Extrahierte SEO-optimierte Texte ---


In [97]:
list_1 = []
for i, text in  enumerate(seo_optimized_texts):
    list_1.append(f"{filtered_urls[i]} optimierter Text: {text}")

In [98]:
cb = Chatbot(systemprompt=f'Sei ein Lektor für SEO optimierte Texte. Du bist spezialisiert auf Grammatikfehler, eingedeutschte Wörter und Rechtschreibfehler', prompt=f"Überprüfe die folgenden Texte auf Fehler und korrigiere diese. Achte ebenfalls auf fehlende Satzzeichen! Verändere nicht die Ausdrucksweise oder die Wortwahl, die Texte sind bereits SEO optimiert. Gebe als Ausgabe die Texte zurück (einschließlich dem link), mit deinen Verbesserungen falls notwendig. Wenn du verbesserungen einfügst, so gebe am Ende ein json dict wieder mit keys:fehler und values:verbesserung. {list_1}")
print(cb.chat())

Natürlich, gerne überprüfe ich die Texte auf Fehler und Satzzeichen. Bitte teile mir die Texte mit, die du überprüft haben möchtest.


# 📥 RAG

In [99]:
"Eine Zahnspange kann Kiefergelenksbeschwerden, Kauen- und Sprechprobleme effektiv behandeln."

"Als in Kenia geborene Kieferorthopädin bringt Dr. Graf eine multikulturelle Perspektive mit und spricht neben Deutsch auch Englisch, Swahili sowie über Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch."

'Als in Kenia geborene Kieferorthopädin bringt Dr. Graf eine multikulturelle Perspektive mit und spricht neben Deutsch auch Englisch, Swahili sowie über Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch.'

In [100]:
pip install langchain faiss-cpu


In [101]:
pip install -U langchain-community

In [102]:
pip install tiktoken

In [103]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# 0) Vector Index (FAISS) initialisieren
#    (Später im Code können wir den Index persistent speichern/neu laden)
# os.environ["OPENAI_API_KEY"] = "DEIN_OPENAI_API_KEY"

embeddings = OpenAIEmbeddings()

# Beispiel-Fehler als "Dokument" für den Vector Store
# "page_content" = Text, "metadata" = beliebige Zusatzinfos
known_error_text = """
Fehler: "Klaren Aligner" wird fälschlicherweise als Eigenname verwendet,
         obwohl es grammatisch richtig "klaren Alignern" sein sollte.

Richtige Anwendung:
- Sagen: "Entdecken Sie die Vorteile von klaren Alignern."
- Oder: "Klare Aligner sind die ..."

Zusätzliche Hinweise:
- Beim Eindeutschen englischer Fachbegriffe auf die Pluralbildung achten.
"""

doc = Document(
    page_content=known_error_text,
    metadata={"error_type": "grammar/de-english", "example_id": "klaren-aligner"}
)

# Vektorindex erzeugen und das "bekannte Fehler"-Dokument ablegen
vector_store = FAISS.from_documents([doc], embeddings)


In [104]:
from langchain.docstore.document import Document

# 1. Neuer Fehler: "Kauen- und Sprechprobleme" statt "Kau- und Sprechprobleme"
doc1_text = """
Fehler: "Eine Zahnspange kann Kiefergelenksbeschwerden, Kauen- und Sprechprobleme effektiv behandeln."
Richtig: "Eine Zahnspange kann Kiefergelenksbeschwerden, Kau- und Sprechprobleme effektiv behandeln."

Grund:
- Falsche Rechtschreibung/Zusammensetzung bei "Kauen-".
- Richtig ist "Kau- und Sprechprobleme".
"""

doc1 = Document(
    page_content=doc1_text,
    metadata={
        "error_type": "grammar/spelling",
        "example_id": "kauen-sprechprobleme"
    }
)

# 2. Neuer Fehler: falsche Formulierung bei Sprachen
doc2_text = """
Fehler: "Als in Kenia geborene Kieferorthopädin ... spricht neben Deutsch auch Englisch, Swahili sowie über Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch."
Richtig: "Als in Kenia geborene Kieferorthopädin ... spricht neben Deutsch auch Englisch und Swahili und verfügt über Grundkenntnisse in Arabisch und Spanisch."

Grund:
- Bessere Formulierung, um 'über Grundkenntnisse' mit 'verfügt über Grundkenntnisse' zu vereinen.
- Straffere und klarere Satzstruktur.
"""

doc2 = Document(
    page_content=doc2_text,
    metadata={
        "error_type": "grammar/style",
        "example_id": "languages-phrase"
    }
)


# Angenommen, du hast bereits:
# embeddings = OpenAIEmbeddings()
# vector_store = FAISS.from_documents([some_initial_docs], embeddings)
#
# -> Dann fügen wir jetzt doc1 und doc2 hinzu:

vector_store.add_documents([doc1, doc2])


['fdcf60d1-fe66-45ba-9868-bc425158f31b',
 'a7f14b27-f7ad-4c1b-ae7a-5b6d3ecbf858']

In [105]:
# faiss_index_path = userdata.get('gdrive_seo_folder') + '/faiss_index'
# vector_store.save_local(faiss_index_path)

In [106]:
# FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)

In [107]:
def chunk_text_2(text, chunk_size=500):
    """
    Beispiel: einfach alle 500 Zeichen ein Chunk.
    Für echte Token-Logik kann man tiktoken oder langchain-Splitter nutzen.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end
    return chunks

chunked_texts = []
for seo_text in seo_optimized_texts:
    # Chunking pro SEO-Text
    text_chunks = chunk_text_2(seo_text, chunk_size=500)
    chunked_texts.append(text_chunks)

# chunked_texts = [
#   [chunk1_of_text1, chunk2_of_text1, ...],
#   [chunk1_of_text2, ...],
#   ...
# ]


In [108]:
from langchain.text_splitter import TokenTextSplitter

def chunk_text_langchain(text, max_tokens=500, overlap=50):
    """
    Teilt den Text anhand der Tokenanzahl auf. Nutzt dafür LangChain's TokenTextSplitter.
    - max_tokens: maximale Tokens pro Chunk
    - overlap: wie viele Tokens Überschneidung zum vorherigen Chunk
    """
    splitter = TokenTextSplitter(
        encoding_name=OpenAIEmbeddings(),  # oder passend zu deinem Modell (z.B. "gpt-3.5-turbo")
        chunk_size=max_tokens,         # maximale Anzahl Tokens pro Chunk
        chunk_overlap=overlap          # Tokens, die sich mit dem vorigen Chunk überschneiden (Kontext)
    )

    chunks = splitter.split_text(text)
    return chunks

# Beispielanwendung:
# seo_text = """Hier Dein langer Text, den du chunken willst ..."""
# chunked = chunk_text_langchain(seo_text, max_tokens=500, overlap=50)
# print(chunked)



chunked_texts = []
for seo_text in seo_optimized_texts:
    # Chunking pro SEO-Text
    chunked = chunk_text_langchain(seo_text, max_tokens=500, overlap=50)
    chunked_texts.append(text_chunks)


In [109]:
chunked_texts

[]

In [110]:
def get_context_from_vector_store(chunk):
    """
    Sucht im FAISS-Index nach passenden Dokumenten zum gegebenen Chunk,
    z. B. bekannte Fehler, die diesem Chunk ähneln.
    """
    # top_k=2 oder so, je nach Bedarf
    results = vector_store.similarity_search(chunk, k=2)
    # results ist eine Liste von Document-Objekten

    # Wir wollen z. B. den Inhalt zusammenfügen als "Kontext":
    context_text = "\n---\n".join([doc.page_content for doc in results])
    return context_text

# Beispielhafte Abfrage pro Chunk
# test_chunk = chunked_texts[0][0]  # Erster Chunk des ersten Textes
# retrieved_context = get_context_from_vector_store(test_chunk)
# print("Kontext aus Vektorindex:\n", retrieved_context)


IndexError: list index out of range

In [ ]:
import json

def proofread_text_with_context(chunk, context):
    """
    Fragt ChatGPT (mittels der Chatbot-Klasse) an, um den Textchunk auf Fehler zu prüfen und zu korrigieren.
    Nutzt den Kontext aus dem Vector Store, um bekannte Fehler zu berücksichtigen.

    Erwartete Antwortstruktur (JSON):

    {
      "corrected_text": "...",
      "new_mistakes_found": [
        {
          "description": "Beschreibung des neuen Fehlers",
          "original_snippet": "Die fehlerhafte Passage"
        },
        ...
      ]
    }
    """

    # 1. System Prompt
    system_prompt = (
        "Du bist ein professioneller Lektor und Grammatik-Experte. "
        "Du kennst deutsche Grammatik, Rechtschreibung und eingedeutschte Fachbegriffe."
    )

    # 2. User Prompt
    #    Wir kombinieren den Kontext und unseren zu prüfenden Text, plus
    #    die Anweisung, nur JSON auszugeben.
    user_prompt = f"""
Im Folgenden siehst du bereits bekannte Fehlerhinweise (Kontext). Nutze diese Infos,
um den Text zu prüfen und zu korrigieren. Solltest du neue Fehler (Grammatik,
falsch eingedeutschte Worte, Satzstellung etc.) finden, liste sie gesondert auf.

Bekannte Fehler (Kontext):
{context}

Text zur Prüfung:
{chunk}

Anweisung:
1) Analysiere den Text gründlich auf sprachliche/grammatische Fehler.
2) Nutze ggf. den Kontext.
3) Korrigiere diese Fehler im Text, ohne den Sinn zu verändern.
4) Liste alle neu gefundenen Fehler (noch nicht im Kontext) zusätzlich auf.
5) Antworte in folgendem JSON-Format (ohne weitere Worte davor oder danach!):

{{
  "corrected_text": "TEXTVERSION KORRIGIERT",
  "new_mistakes_found": [
    {{
      "description": "Beschreibung des Fehlers",
      "original_snippet": "Snippet der Original-Passage"
    }}
  ]
}}
"""

    # 3. Chatbot verwenden:
    cb = Chatbot(systemprompt=system_prompt, prompt=user_prompt)

    # Da wir keine Streaming-Ausgabe brauchen, nutzen wir hier `chat()` statt `chat_with_streaming()`.
    response_raw = cb.chat()

    # 4. JSON parsen
    try:
        parsed = json.loads(response_raw)
        # parsed = {
        #   "corrected_text": "...",
        #   "new_mistakes_found": [...]
        # }
        return parsed

    except json.JSONDecodeError:
        print("Fehler: ChatGPT hat kein gültiges JSON zurückgegeben.")
        return {
            "corrected_text": "Fehler: Keine gültige JSON-Antwort.",
            "new_mistakes_found": []
        }


In [ ]:
all_corrected_texts = []
all_new_mistakes = []

for text_chunks in chunked_texts:  # => Jede Liste von Chunks (pro SEO-Text)
    corrected_text_chunks = []

    for chunk in text_chunks:
        # 3a) Kontext abfragen
        context = get_context_from_vector_store(chunk)


        # 4a) Prompt ChatGPT (Korrektur)
        result = proofread_text_with_context(chunk, context)

        corrected_text = result["corrected_text"]
        new_mistakes = result["new_mistakes_found"]

        # Sammeln
        corrected_text_chunks.append(corrected_text)
        all_new_mistakes.extend(new_mistakes)

    # Pro SEO-Text fügen wir die korrigierten Chunks zusammen.
    full_corrected_text = "\n".join(corrected_text_chunks)
    all_corrected_texts.append(full_corrected_text)

# Jetzt haben wir:
# all_corrected_texts = [ "korrigierter SEO Text Nr.1", "korrigierter SEO Text Nr.2", ...]
# all_new_mistakes = Liste aller neu gefundenen Fehler


In [ ]:
for _ in all_corrected_texts:
  print(_)

In [ ]:
all_new_mistakes